In [1]:
from IPython.display import clear_output
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from typing import List

## 모델 진행상황 결과 확인하는 라이브러리
from tqdm.notebook import tqdm

## Huggingface 라이브러리 받아오는 코드
!pip install transformers datasets tokenizers sentencepiece x-transformers pytorch-lightning wandb
!wget https://raw.githubusercontent.com/monologg/KoBERT-Transformers/master/kobert_transformers/tokenization_kobert.py
from transformers import DistilBertModel
from tokenization_kobert import KoBertTokenizer

## 기타 사용할 머신러닝 알고리즘
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

clear_output()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_path = '/content/drive/MyDrive/의현/018_감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training.zip'
val_path = '/content/drive/MyDrive/의현/018_감성대화/Validation_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Validation.zip'
!cp -r "$train_path" ./
!cp -r "$val_path" ./
drive.flush_and_unmount()

In [4]:
!unzip './감성대화말뭉치(최종데이터)_Training.zip'
!unzip './감성대화말뭉치(최종데이터)_Validation.zip'
clear_output()

In [4]:
import os
import shutil
from sys import platform
from glob import glob

train_data_path = './감성대화말뭉치(최종데이터)_Training.xlsx'
val_data_path = './감성대화말뭉치(최종데이터)_Validation.xlsx'

if platform == "linux" or platform == "linux2":
    pass
elif platform == "darwin":
    train_data_path = os.path.join("dset", train_data_path)
    val_data_path = os.path.join("dset", val_data_path)

In [53]:
import numpy as np
import pandas as pd

train_dataset = pd.read_excel(train_data_path, index_col = 'Unnamed: 0')
val_dataset = pd.read_excel(val_data_path, index_col = 'Unnamed: 0')

In [54]:
train_dataset[['사람문장1', '사람문장2', '사람문장3']] = train_dataset[['사람문장1', '사람문장2', '사람문장3']].fillna('').astype(str)
train_dataset = pd.melt(train_dataset, id_vars=['연령', '성별', '신체질환', '감정_대분류', '감정_소분류'], value_vars=['사람문장1', '사람문장2', '사람문장3'], value_name='sentence')
train_dataset = train_dataset[train_dataset['sentence'] != '']
train_dataset = train_dataset.drop(['variable'], axis=1)
train_dataset

,연령,성별,신체질환,감정_대분류,감정_소분류,sentence
0,청년,여성,해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,청년,여성,해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,청년,여성,해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,청년,여성,해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,청년,여성,해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.
...,...,...,...,...,...,...
154885,노년,남성,만성질환 무,분노,성가신,노년층을 위한 경제적 지원이나 부업 같은 것도 알아보아야겠어.
154886,노년,여성,만성질환 무,불안,초조한,남편과 함께 게이트볼이나 치러 가야겠어. 그럼 기분이 나아질 것 같아.
154887,노년,여성,만성질환 무,상처,희생된,남편과 함께 실버 일자리나 노년층을 위한 국가 지원에 대해 자세히 알아보아야겠어.
154888,노년,여성,만성질환 무,불안,걱정스러운,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.


In [55]:
val_dataset[['사람문장1', '사람문장2', '사람문장3']] = val_dataset[['사람문장1', '사람문장2', '사람문장3']].fillna('').astype(str)
val_dataset = pd.melt(val_dataset, id_vars=['연령', '성별', '신체질환', '감정_대분류', '감정_소분류'], value_vars=['사람문장1', '사람문장2', '사람문장3'], value_name='sentence')
val_dataset = val_dataset[val_dataset['sentence'] != '']
val_dataset = val_dataset.drop(['variable'], axis=1)
val_dataset

,연령,성별,신체질환,감정_대분류,감정_소분류,sentence
0,청년,남성,해당없음,불안,두려운,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...
1,청년,남성,해당없음,불안,두려운,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.
2,청년,남성,해당없음,불안,두려운,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.
3,청년,남성,해당없음,불안,두려운,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.
4,청년,남성,해당없음,불안,두려운,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.
...,...,...,...,...,...,...
19838,청년,여성,해당없음,분노,분노,말해봐도 소용이 없어.
19839,중년,여성,해당없음,기쁨,감사하는,부모님께는 자주 찾아뵙는 게 우선일 거고 남편에겐 힘낼 수 있는 말을 자주 해 줄 ...
19858,청소년,여성,해당없음,불안,불안,좋아하는 노래를 한 곡 듣고 기분이 나아진 상태에서 친구들과 대화를 하면 좋을 것 같아.
19860,청소년,여성,해당없음,기쁨,편안한,함께 점심도 먹고 친하게 지내고 있어.


In [56]:
MAX_LEN = 64
TRAIN_BATCH_SIZE = 2048
VALID_BATCH_SIZE = 2048
tokenizer = KoBertTokenizer.from_pretrained('monologg/distilkobert')
clear_output()

In [57]:
from sklearn.model_selection import train_test_split

train_size = 0.50

val_train, val_dataset = train_test_split(val_dataset, train_size = train_size, stratify=val_dataset['감정_대분류'], random_state=77)
train_dataset = pd.concat([train_dataset, val_train], axis = 0)

In [58]:
train_size = 0.95

train_dataset, test_dataset = train_test_split(train_dataset, train_size = train_size, stratify=train_dataset['감정_대분류'], random_state=77)
train_dataset = train_dataset.reset_index(drop=True)
test_dataset = test_dataset.reset_index(drop=True)
val_dataset = val_dataset.reset_index(drop=True)

In [59]:
X_train = train_dataset.drop('감정_대분류', axis = 1)
y_train = train_dataset['감정_대분류']
X_test = test_dataset.drop('감정_대분류', axis = 1)
y_test = test_dataset['감정_대분류']
X_val = val_dataset.drop('감정_대분류', axis = 1)
y_val = val_dataset['감정_대분류']

In [60]:
le = preprocessing.LabelEncoder()

train_labels = le.fit_transform(y_train.values)

In [61]:
num_unique_labels = len(le.classes_)
print("독립적인 레이블의 개수:", num_unique_labels)

독립적인 레이블의 개수: 6


In [62]:
class Dataset_Generation:
    def __init__(self, df, labels, tokenizer, max_len):
        self.len = len(df)
        self.data = df
        self.tokenizer = tokenizer
        self.labels = labels
        self.max_len = max_len

    def __getitem__(self, idx):
        title = ", ".join(str(_) for _ in self.data.loc[idx, :].values)
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding = 'max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [63]:
train_dataset = X_train
test_dataset = X_test
val_dataset = X_val
train_dataset = train_dataset.reset_index(drop=True)
test_dataset = test_dataset.reset_index(drop=True)
val_dataset = val_dataset.reset_index(drop=True)

print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))
print("VAL Dataset: {}".format(val_dataset.shape))

test_labels = le.transform(y_test.values)
val_labels = le.transform(y_val.values)
train_set = Dataset_Generation(train_dataset, train_labels, tokenizer, MAX_LEN)
test_set = Dataset_Generation(test_dataset, test_labels, tokenizer, MAX_LEN)
val_set = Dataset_Generation(val_dataset, val_labels, tokenizer, MAX_LEN)

TRAIN Dataset: (147192, 5)
TEST Dataset: (7747, 5)
VAL Dataset: (8984, 5)


In [64]:
first_item = train_set[0]

for key, value in first_item.items():
    print(f"{key}: {value}")

ids: tensor([   2, 1476, 5712,   46, 1419,   46, 1931, 6573, 7352, 2095,   46,  517,
        5499, 7350, 1375,   46, 3672, 4599, 2224,  862,  936, 5947, 5562, 5211,
        5920, 5702,   54,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1])
mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
targets: 2


In [65]:
decoded_text = tokenizer.decode(first_item['ids'])
print(decoded_text)

[CLS] 노년, 남성, 만성질환 무, 구역질 나는, 이런 취급 받는 거 견디기가 힘드네.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]


In [66]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 8,
                'pin_memory' :True
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'num_workers': 8,
                'pin_memory' :True
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'num_workers': 8,
                'pin_memory' :True
                }

train_loader = DataLoader(train_set, **train_params)
test_loader = DataLoader(test_set, **test_params)
val_loader = DataLoader(val_set, **val_params)

In [67]:
from typing import Any
import pytorch_lightning as pl
from transformers import get_scheduler
from torch.optim import AdamW, Optimizer

class BERTLightningModule(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.save_hyperparameters()
        self.l1 = DistilBertModel.from_pretrained('monologg/distilkobert')
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.classifier = torch.nn.Linear(768, num_unique_labels)
        self.loss_function = torch.nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        output = self.classifier(pooler)
        return output

    def training_step(self, batch, batch_idx):
        ids = batch['ids']
        mask = batch['mask']
        targets = batch['targets']

        outputs = self(ids, mask)
        loss = self.loss_function(outputs, targets)

        self.log('train_loss', loss, prog_bar = True)
        self.log('train_acc', self.calculate_accuracy(outputs, targets), prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        ids = batch['ids']
        mask = batch['mask']
        targets = batch['targets']

        outputs = self(ids, mask)
        loss = self.loss_function(outputs, targets)

        self.log('test_loss', loss, prog_bar = True)
        self.log('test_acc', self.calculate_accuracy(outputs, targets), prog_bar=True)

    def validation_step(self, batch, batch_idx):
        ids = batch['ids']
        mask = batch['mask']
        targets = batch['targets']

        outputs = self(ids, mask)
        loss = self.loss_function(outputs, targets)

        self.log('val_loss', loss, prog_bar = True)
        self.log('val_acc', self.calculate_accuracy(outputs, targets), prog_bar=True)

    def configure_optimizers(self) -> tuple[list[Optimizer], list[dict[str, Any]]]:
        do_decay = [p for p in self.parameters() if p.requires_grad and p.ndim >= 2]
        no_decay = [p for p in self.parameters() if p.requires_grad and p.ndim < 2]
        param_groups = [{"params": do_decay}, {"params": no_decay, "weight_decay": 0.0}]

        optimizer = AdamW(param_groups, **self.config.optim.optimizer)
        scheduler = get_scheduler(optimizer=optimizer, **self.config.optim.scheduler)
        return [optimizer], [{"scheduler": scheduler, "interval": "step"}]

    @staticmethod
    def calculate_accuracy(preds, targets):
        _, predicted = torch.max(preds, dim=1)
        correct = torch.sum(predicted == targets)
        acc = correct.float() / targets.size(0)
        return acc

    def train_dataloader(self):
        return train_loader

    def test_dataloader(self):
        return test_loader

    def val_dataloader(self):
        return val_loader

In [69]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

checkpoint = ModelCheckpoint(
    monitor="val_loss", mode="min", save_weights_only=True
)

trainer = Trainer(
    accelerator="gpu",
    devices="auto",
    precision=16,
    log_every_n_steps=50,
    max_epochs=50,
    gradient_clip_val=0,
    accumulate_grad_batches=1,
    val_check_interval=1.0,
    callbacks=[checkpoint, LearningRateMonitor("step")],
)

from easydict import EasyDict

config = EasyDict({})
config.optim = {}
config.optim.optimizer = {}
config.optim.optimizer.lr = 1e-4
config.optim.optimizer.eps = 1e-6
config.optim.optimizer.weight_decay = 1e-6
config.optim.scheduler = {}
config.optim.scheduler.name = "linear"
config.optim.scheduler.num_warmup_steps = 1500
config.optim.scheduler.num_training_steps = 15000

trainer.fit(BERTLightningModule(config), train_loader, val_loader)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
Some weights of the model checkpoint at monologg/distilkobert were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model f

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [70]:
trainer.test(ckpt_path=checkpoint.best_model_path, dataloaders=[val_loader])

INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_16/checkpoints/epoch=7-step=576.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_16/checkpoints/epoch=7-step=576.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9692786931991577     │
│         test_loss         │   0.046426620334386826    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.046426620334386826, 'test_acc': 0.9692786931991577}]

In [71]:
trainer.test(ckpt_path=checkpoint.best_model_path, dataloaders=[test_loader])

INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_16/checkpoints/epoch=7-step=576.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_16/checkpoints/epoch=7-step=576.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9713437557220459     │
│         test_loss         │   0.047236569225788116    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.047236569225788116, 'test_acc': 0.9713437557220459}]

In [72]:
checkpoint.best_model_path

'/content/lightning_logs/version_16/checkpoints/epoch=7-step=576.ckpt'

In [74]:
example = '청소년, 남성, 가족관계, 대인관계, 버려진, 집에 혼자 있으면 외롭고 우울해.'

# 토큰화 및 인코딩
tokens = tokenizer.encode_plus(
    example,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

model = BERTLightningModule.load_from_checkpoint(checkpoint.best_model_path)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

with torch.no_grad():
    outputs = model(input_ids, attention_mask)

predictions = torch.argmax(outputs, dim=1)

print('\n##############')
print('결과 :')
print(le.inverse_transform(predictions.cpu().numpy()))

Some weights of the model checkpoint at monologg/distilkobert were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



##############
결과 :
['상처']
